# Z Estimator

## Importing libraries

In [1]:
# Libraries to help with reading and manipulating data
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

## Extracting the dataset

In [2]:
df_train_prop2016 = pd.read_csv('properties_2016.csv')
df_train_prop2016.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [ ]:
df_train_prop2016.shape
print(r'The dataset has {rownum} rows and {colnum} columns'.format(rownum = df_train_prop2016.shape[0], colnum = df_train_prop2016.shape[1]))

These columns are going to be removed
<ol>
<li>calculatedbathnbr: it's not relevant like bathroomcnt, practically both have same values</li>
<li>decktypeid: got only 1 value = 66 (1%)</li>
<li>fips: country code</li>
<li>fireplaceflag: 99% is missing, and we can use fireplacecnt</li>
</ol>

In [3]:
df_train_prop2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 58 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   parcelid                      int64  
 1   airconditioningtypeid         float64
 2   architecturalstyletypeid      float64
 3   basementsqft                  float64
 4   bathroomcnt                   float64
 5   bedroomcnt                    float64
 6   buildingclasstypeid           float64
 7   buildingqualitytypeid         float64
 8   calculatedbathnbr             float64
 9   decktypeid                    float64
 10  finishedfloor1squarefeet      float64
 11  calculatedfinishedsquarefeet  float64
 12  finishedsquarefeet12          float64
 13  finishedsquarefeet13          float64
 14  finishedsquarefeet15          float64
 15  finishedsquarefeet50          float64
 16  finishedsquarefeet6           float64
 17  fips                          float64
 18  fireplacecnt          

In [11]:
df_train_2016 = pd.read_csv('train_2016_v2.csv')
df_train_2016.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [20]:
df_train_2016[df_train_2016[['parcelid', 'transactiondate']].duplicated()]

,parcelid,logerror,transactiondate


## Exploratory Data Analysis

In [8]:
# profile = ProfileReport(
#     df_train_prop2016, 
#     title = "Profiling Report - Train Dataset",
#     dataset={
#         "description": "Profiling Report for train dataset",
#         "author": "Nestor Arias",
#         "copyright_holder": "AI Caramba !!",
#         "copyright_year": 2024
#     }
# )
# profile.to_notebook_iframe()